In [ ]:
import os
import pandas as pd
import geopandas as gpd
import folium
from shapely import wkt

from urbantrips.carto import stops
from urbantrips.utils import utils
from urbantrips.datamodel import services

In [ ]:
def manual_change_in_node_coords(stops,id_linea,poligon_wkt,node_coords = False):
    poligon = wkt.loads(poligon_wkt)
    print(f"Cambiando stops para ID linea {id_linea}")
    temp_stops_line_stops = stops.loc[stops.id_linea==id_linea,:]

    temp_stops_line_stops = gpd.GeoDataFrame(temp_stops_line_stops,
                                 geometry = gpd.GeoSeries.from_xy(
                                     x=temp_stops_line_stops.stop_x,
                                     y=temp_stops_line_stops.stop_y, crs='EPSG:4326'),
                                 crs='EPSG:4326')

    stops_to_change = temp_stops_line_stops.loc[temp_stops_line_stops.geometry.map(lambda g: g.intersects(poligon)),:]
    print("Cantidad de paradas a cambiar",len(stops_to_change))

    ids_stops_to_change = stops_to_change.index
    stops_to_keep = stops.loc[~stops.index.isin(ids_stops_to_change),:]
    print("Cantidad de paradas que permanecen igual",len(stops_to_keep))

    # split 
    node_id = stops_to_change.node_id.iloc[0]
    change_node_id = {k:node_id for k in stops_to_change.node_id}
    stops_to_change.loc[:,['node_id']] = stops_to_change['node_id'].replace(change_node_id)

     
    # If a set of node coords are given 
    if node_coords:
        stops_to_change.loc[:, 'node_x'] = node_coords[0]
        stops_to_change.loc[:, 'node_y'] = node_coords[1]

    else:
        # Creat a latlong using average stops coords for that node
        x_new_long = stops_to_change.groupby('node_id').apply(
            lambda df: df.stop_x.mean()).to_dict()
        y_new_long = stops_to_change.groupby('node_id').apply(
            lambda df: df.stop_y.mean()).to_dict()

        stops_to_change.loc[:, 'node_x'] = stops_to_change['node_id'].replace(x_new_long)
        stops_to_change.loc[:, 'node_y'] = stops_to_change['node_id'].replace(y_new_long)



    stops_to_change = stops_to_change.drop('geometry',axis=1)
    new_stops = pd.concat([stops_to_keep,stops_to_change])
    
    linea_data = new_stops.loc[new_stops.id_linea==id_linea,:]
    changed_stops_gdf = gpd.GeoDataFrame(linea_data,
                                 geometry = gpd.GeoSeries.from_xy(
                                     x=linea_data.node_x,
                                     y=linea_data.node_y, crs='EPSG:4326'),
                                 crs='EPSG:4326')
    return new_stops, changed_stops_gdf
    

In [ ]:
URBANTRIPS_PATH = "[PATH]"

os.chdir(URBANTRIPS_PATH)

configs = utils.leer_configs_generales()
conn_data = utils.iniciar_conexion_db(tipo='data')
conn_insumos = utils.iniciar_conexion_db(tipo='insumos')

Este notebook sirve para una vez corrido el proceso `services.process_services()` analizar los resultados y evaluar si hubo errores en la clasificación de servicios. Esto puede deberse a movimientos herráticos del vehículo pero también a una disposición de paradas (y los nodos a los cuales están asignadas). Pueden existir recorridos de colectivos que vayan y vuelvan por una misma calle o una calle paralela pero en un sentido en un momento y en otro sentido en otro (importante recordar que los recorridos tomados aquí son de sentido único tal como se explica en el notebook  `stops_creation_with_node_id_helper`). 

Este notebook permite introducir modificaciones en esas paradas, simplificandolas en un mismo `node` de modo que la traza de puntos gps no marque un cambio de sentido en el paso de paradas. 


# 1. Detectar el problema

Se elegiriá trabajar con una linea por vez. Primero se obtienen los servicios y los registros de esa linea (puntos gps y paradas).

In [ ]:
query = "select * from services_stats order by servicios_originales_sin_dividir limit 5"
services_stats = pd.read_sql(query, conn_data)
services_stats

In [ ]:
selected_line_id = 1

In [ ]:
line_stops_gdf = pd.read_sql(f"select * from stops where id_linea = {selected_line_id}", conn_insumos)
# use only nodes as stops
line_stops_gdf = line_stops_gdf.drop_duplicates(subset = ['id_linea','id_ramal','node_id'])
line_stops_gdf = gpd.GeoDataFrame(line_stops_gdf,
                             geometry = gpd.GeoSeries.from_xy(
                                 x=line_stops_gdf.node_x, y=line_stops_gdf.node_y, crs='EPSG:4326'),
                              crs='EPSG:4326'
                             )
line_stops_gdf = line_stops_gdf.to_crs(epsg=configs['epsg_m'])
branches = line_stops_gdf.id_ramal.unique()

q = f"""
select dia,id_linea,id_ramal,interno, DATETIME(fecha,'unixepoch') as fecha,latitud,longitud,service_type,distance_km,h3 
from gps 
where id_linea = {selected_line_id}
order by dia, id_linea, interno, fecha 

"""

gps_points = pd.read_sql(q, conn_data)    
gps_points = gpd.GeoDataFrame(gps_points,
                             geometry = gpd.GeoSeries.from_xy(
                                 x=gps_points.longitud, y=gps_points.latitud, crs='EPSG:4326'),
                              crs='EPSG:4326'
                             )
gps_points = gps_points.to_crs(epsg=configs['epsg_m'])


Aquí se pueden ver las paradas de cada ramal de la línea.

In [ ]:
line_stops_gdf.explore(column = 'id_ramal',
              tiles="CartoDB positron",categorical = True,
              cmap = 'tab10',
              marker_kwds = {'radius':10})

Se clasifican los servicios utilizando estas mismas paradas. Estos servicios no se subiran a la base de datos por ahora. Solo se utilizarán en el notebook.

In [ ]:
gps_points_with_new_service_id = gps_points\
        .groupby(['dia', 'interno'], as_index=False)\
        .apply(services.classify_line_gps_points_into_services,
               line_stops_gdf=line_stops_gdf)\
        .droplevel(0)

Se puede elegir un interno de todos los que operaron ese día.

In [ ]:
gps_points_with_new_service_id.reindex(columns = ['dia','interno']).drop_duplicates().sample(10)

Se eelecciona el dia e interno que se quiere analizar y se observa dentro de cada servicio original (tal cual lo declara el conductor en la tabla gps) cuantos servicios nuevos fueron creados.   


In [ ]:
selected_day = '2022-11-09'
selected_vehicle = 1234

basic_cols = ['id_ramal','fecha', 'majority', 'change','original_service_id','new_service_id', 'idling', 'service_id','geometry']
b = [[f'order_{b}',f'distance_to_stop_{b}', f'temp_change_{b}', f'consistent_{b}',f'change_{b}'] for b in branches]
branches_cols = [item for sublist in b for item in sublist]
cols = basic_cols + branches_cols 
mask = (gps_points_with_new_service_id.interno ==  selected_vehicle) & (gps_points_with_new_service_id.dia == selected_day)

vehicle_gps = gps_points_with_new_service_id.loc[mask,cols]

pd.crosstab(vehicle_gps.original_service_id,vehicle_gps.service_id)

Puede elegirse el id de un servicio original tal cual se informaba en los datos gps para ver en cuántos servicios se subdividio y dónde.

In [ ]:
selected_original_service = 1
service_gps = vehicle_gps.loc[vehicle_gps.original_service_id == selected_original_service,:]

La siguiente visualización muestra ese servicio original con colores para cada servicio nuevo, junto con las paradas de la linea. Puede elegir entre capas con el selector ubicado arriba a la derecha. Se pueden dónde hay cambios de servicio que no están bien hecho, producto de las zonas donde hay paradas con muchos cambios de sentido que mejor convenga simplificar en un único `node`.

In [ ]:
m = line_stops_gdf.explore(column = 'id_ramal',
              tiles="CartoDB positron",categorical = True,
              cmap = 'tab10',
              marker_kwds = {'radius':3}, name = 'Paradas')

service_gps.explore(m=m,column = 'service_id',
              tiles="CartoDB positron",categorical = True,
              cmap = 'tab10',
              marker_kwds = {'radius':10}, name = 'GPS')

if service_gps.change.any():
    service_gps.query("change==True").explore(m=m,color = 'red',
                  tiles="CartoDB positron",
                  cmap = 'tab10',
                  marker_kwds = {'radius':10}, name = 'Service id change')

folium.LayerControl().add_to(m)

m

## Debuggin

Si el comportamiento no es el esperado si se quiere ver la referenciación de cada punto gps en los nodos de cada ramal, se puede correr la funcion que asigna el servicio para un interno y para un id de servicio original. Esto dejara las variables insumo utilizadas para evaluar un cambio en el orden de paso 

In [ ]:
mask = (gps_points.interno ==  selected_vehicle)\
& (gps_points.dia == selected_day)

debug_vehicle_data = gps_points.loc[mask,:]


original_service_id = debug_vehicle_data\
    .reindex(columns=['dia', 'interno', 'service_type'])\
    .groupby(['dia', 'interno'])\
    .apply(services.create_original_service_id)
original_service_id = original_service_id.service_type
original_service_id = original_service_id.droplevel([0, 1])
debug_vehicle_data['original_service_id'] = original_service_id


debug_service_data = debug_vehicle_data.loc[debug_vehicle_data.original_service_id == selected_original_service,:]

# run function on service to debug
debug_service_data = services.infer_service_id_stops(debug_service_data,line_stops_gdf, debug=True)

# 2. Modificar las paradas en una zona a definir

Puede introducirse un polígono en formato WKT para fusionar todas esas paradas en un mismo nodo y visualizar qué paradas afectaría. Para eso se puede usar un visualizador online para dibujar un polígono y obtener el WKT como [Cercalia](http://www.cercalia.com/api/v5/examples/digitalize.html). Tambien se puede proveer un par de coordenadas (formato xy) para el nuevo nodo, o dejarlo vacío y simplemente tomará el promedio de las paradas a modificar.

In [ ]:
poligon_wkt = 'POLYGON((-58.5573841856895 -34.566332360730115,-58.52528350819925 -34.56612032349501,-58.52528350819925 -34.578912268933784,-58.556525878804734 -34.578629595274634,-58.5573841856895 -34.566332360730115))'
new_node_coord = (-58.536492,-34.567169)

Puede chequearse que ese polígono contenga las paradas que se desea modificar:

In [ ]:
m = line_stops_gdf.explore(column = 'id_ramal',
              tiles="CartoDB positron",categorical = True,
              cmap = 'tab10',
              marker_kwds = {'radius':3}, name = 'Paradas')

service_gps.explore(m=m,column = 'service_id',
              tiles="CartoDB positron",categorical = True,
              cmap = 'tab10',
              marker_kwds = {'radius':10}, name = 'GPS')

poligon_series = gpd.GeoSeries(wkt.loads(poligon_wkt), crs='EPSG:4326')
poligon_series.explore(m=m,color = '#00000077', name='Poligono')

if new_node_coord:
    folium.Marker(new_node_coord[::-1], name = 'Nuevo Nodo').add_to(m)
    
folium.LayerControl().add_to(m)
m

A continuación se lee el archivo csv de paradas que utiliza `urbantrips` para producir la tabla `stops`. Este archivo será modificado en este notebook las veces que sea necesaria, para cada linea y cada intervencion con un polígono puntual. Luego si asi se desea será guardado con el mismo nombre, u otro (con lo cual habrá que cambiarlo en `configuraciones_generales.yaml`). Si el archivo sufre modificaciones, habrá que correr nuevamente el proceso que crea la tabla `stops` corriendo en este mismo notebook `stops.create_stops_table()`.

In [ ]:
stops_path = os.path.join("data","data_ciudad",configs['nombre_archivo_paradas'])
original_stops = pd.read_csv(stops_path)
print("Cantidad de paradas",len(original_stops))

In [ ]:
modified_stops, stops_to_change_gdf = manual_change_in_node_coords(
    stops = original_stops,
    id_linea = selected_line_id,
    poligon_wkt = poligon_wkt,
    node_coords = new_node_coord)
print('Cantidad de paradas modificadas',len(modified_stops))

Al convertir estas nuevas en un GeoDataFrame utilizable para clasificar servicios, se puede volver a clasificar los servicios de ese vehículo para ese día. 

In [ ]:
stops_to_change_gdf = stops_to_change_gdf\
    .drop_duplicates(subset = ['id_linea','id_ramal','node_id'])\
    .to_crs(epsg=configs['epsg_m'])

mask = (gps_points.interno ==  selected_vehicle) & (gps_points.dia == selected_day)

new_services = gps_points.loc[mask,:]\
        .groupby(['dia', 'interno'], as_index=False)\
        .apply(services.classify_line_gps_points_into_services,
               line_stops_gdf=stops_to_change_gdf)\
        .droplevel(0)

Ahora al observar cada servicio original y cuantas veces fue segmentado en nuevos servicios, se debería percibir un cambio

In [ ]:
pd.crosstab(new_services.original_service_id,new_services.service_id)

In [ ]:
new_service_gps = new_services.loc[new_services.original_service_id == selected_original_service,:]
new_service_gps.service_id.value_counts()

Se puede volver a visualizar:

In [ ]:
m = stops_to_change_gdf.explore(column = 'id_ramal',
              tiles="CartoDB positron",categorical = True,
              cmap = 'tab10',
              marker_kwds = {'radius':3}, name = 'Paradas')

new_service_gps.explore(m=m,column = 'service_id',
              tiles="CartoDB positron",categorical = True,
              cmap = 'tab10',
              marker_kwds = {'radius':10}, name = 'GPS')

# this is completely optional
folium.LayerControl().add_to(m)

m

Si existe otro punto problemático, puede replicarse el paso 2 aquí las veces que sea necesario, estableciendo un nuevo poligono de intervención, un nuevo par de coordenadas para el nodo y volver a correr  `manual_change_in_node_coords()` sobre el dataset de paradas ya intervenido previamente.

Eventualmente, puede crearse un `yaml` con todas los cambios manuales con la siguiente estructura y luego iterar una función que haga los cambios, de modo que queden documentados:

```yaml
# un esquema es por id linea 

1: 
    - poligon_wkt: 'POLYGON((xxxyyy))'
      new_node_coord: (xxx,yyyy)
      
    - poligon_wkt: 'POLYGON((xxxyyy))'
      new_node_coord: (xxx,yyyy)
2:
    - poligon_wkt: 'POLYGON((xxxyyy))'
      new_node_coord: (xxx,yyyy)
      
    - poligon_wkt: 'POLYGON((xxxyyy))'
      new_node_coord: (xxx,yyyy)
```

# 3. Guardar los resultados 

Por un lado es necesario volver a guardar el csv. Se puede usar el mismo `stops_path` o un nuevo nombre de archivo (que deberá corregirse en `configuraciones_generales.yaml`.

In [ ]:
modified_stops_path = ""
modified_stops.to_csv(modified_stops_path)

Como las paradsa han sido modificadas, es necesario volver a subir las paradas con la funcion `stops.create_stops_table()`

In [ ]:
#stops.create_stops_table()

Luego es momento de volver a borrar los servicios para esta linea cuyos servicios fueron clasificados con un set de paradas que no era el mejor. Para eso utilizar `delete_services_data(id_linea)`

In [ ]:
#services.delete_services_data(selected_line_id)

Ahora ya estamos en condiciones de volver a correr el proceso de clasificacion de servicios con `services.process_services()`

In [ ]:
#services.process_services()